In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf

import random
import pickle
import os

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from helper import cal_metrics, f_get_minibatch
from sklearn.neural_network import MLPClassifier

from import_data import *
import scipy

from model import SEFS_S_Phase, SEFS_SS_Phase

In [ ]:
p         = 10
blocksize = 10  #overall, p*blocksize

sigma_n = 1.0
seed    = 1234

max_labeled_samples   = 10
max_unlabeled_samples = 2000

model_name = 'proposed_mvBern' 

DATASET_PATH = 'TWOMOON/ns_{}nu_{}'.format(int(2*max_labeled_samples), max_unlabeled_samples)

In [ ]:
OUT_ITERATION = 100

RESULTS  = np.zeros([OUT_ITERATION, 2])
RESULTS2 = np.zeros([OUT_ITERATION, 2])

In [ ]:
out_itr = 0

save_path  = './workspace/{}/itr{}/'.format(DATASET_PATH, out_itr)
if not os.path.exists(save_path):
    os.makedirs(save_path)

seed    = 1234    
seed    = seed + out_itr * 4

tr_X, tr_Y, tr_Y_onehot = get_noisy_two_moons(n_samples=1000, n_feats=p, noise_twomoon=0.1, noise_nuisance=sigma_n, seed_=seed)
UX, UY, UY_onehot       = get_noisy_two_moons(n_samples=1000, n_feats=p, noise_twomoon=0.1, noise_nuisance=sigma_n, seed_=seed+1)
va_X, va_Y, va_Y_onehot = get_noisy_two_moons(n_samples=1000, n_feats=p, noise_twomoon=0.1, noise_nuisance=sigma_n, seed_=seed+2)
te_X, te_Y, te_Y_onehot = get_noisy_two_moons(n_samples=1000, n_feats=p, noise_twomoon=0.1, noise_nuisance=sigma_n, seed_=seed+3)


block_noise = 0.3
tr_X = get_blockcorr(tr_X, blocksize, block_noise, seed)
UX   = get_blockcorr(UX, blocksize, block_noise, seed+1)
va_X = get_blockcorr(va_X, blocksize, block_noise, seed+2)
te_X = get_blockcorr(te_X, blocksize, block_noise, seed+3)



random.seed(seed)
idx1 = random.sample(np.where(tr_Y==1)[0].tolist(), max_labeled_samples)
idx0 = random.sample(np.where(tr_Y==0)[0].tolist(), max_labeled_samples)

idx  = idx1 + idx0
random.shuffle(idx)

tr_X        = tr_X[idx]
tr_Y        = tr_Y[idx]
tr_Y_onehot = tr_Y_onehot[idx]

tr_X_org = np.copy(tr_X)
va_X_org = np.copy(va_X)
te_X_org = np.copy(te_X)
UX_org   = np.copy(UX)

scaler = MinMaxScaler()
scaler.fit(np.concatenate([tr_X, UX], axis=0))


tr_X    = scaler.transform(tr_X)
va_X    = scaler.transform(va_X)
te_X    = scaler.transform(te_X)

if max_unlabeled_samples > 1000:
    UX_, UY_, UY_onehot_       = get_noisy_two_moons(n_samples=1000, n_feats=p, noise_twomoon=0.1, noise_nuisance=sigma_n, seed_=seed-1)
    UX_                        = get_blockcorr(UX_, blocksize, block_noise, seed-1)
    
    UX        = np.concatenate([UX, UX_], axis=0)
    UY        = np.concatenate([UY, UY_])
    UY_onehot = np.concatenate([UY_onehot, UY_onehot_], axis=0)
else:
    UX        = UX[:max_unlabeled_samples]
    UY        = UY[:max_unlabeled_samples]
    UY_onehot = UY_onehot[:max_unlabeled_samples]    
UX      = scaler.transform(UX)       

### Multivariate Bernoulli Generation

In [ ]:
cov = np.corrcoef(UX.T)
remove_idx = []

cov_new    = np.delete(cov, remove_idx, axis=0)
cov_new    = np.delete(cov_new, remove_idx, axis=1)

L          = scipy.linalg.cholesky(cov_new, lower=True)

cov_new = []
cov     = []


def mask_generation(mb_size_, pi_):
    '''
        Phi(x; mu, sigma) = 1/2 * (1 + erf( (x-mu)/(sigma * sqrt(2)) )) 
        --> Phi(x; 0,1)   = 1/2 * (1 + erf( x/sqrt(2) )) 
    '''
    if len(remove_idx) == 0:
        epsilon = np.random.normal(loc=0., scale=1., size=[np.shape(L)[0], mb_size_])
        g       = np.matmul(L, epsilon)
    else:
        present_idx = [i for i in range(x_dim) if i not in remove_idx]
        epsilon     = np.random.normal(loc=0., scale=1., size=[np.shape(L)[0], mb_size_])
        g2      = np.random.normal(loc=0., scale=1., size=[len(remove_idx), mb_size_])
        g1      = np.matmul(L, epsilon)
        g       = np.zeros([x_dim, mb_size_])

        g[present_idx, :] = g1
        g[remove_idx, :]  = g2

    m = (1/2 * (1 + scipy.special.erf(g/np.sqrt(2)) ) < pi_).astype(float).T    
    return m


def copula_generation(mb_size_):
    if len(remove_idx) == 0:
        epsilon = np.random.normal(loc=0., scale=1., size=[np.shape(L)[0], mb_size_])
        g       = np.matmul(L, epsilon)
    else:
        present_idx = [i for i in range(x_dim) if i not in remove_idx]
        epsilon     = np.random.normal(loc=0., scale=1., size=[np.shape(L)[0], mb_size_])
        g2      = np.random.normal(loc=0., scale=1., size=[len(remove_idx), mb_size_])
        g1      = np.matmul(L, epsilon)
        g       = np.zeros([x_dim, mb_size_])

        g[present_idx, :] = g1
        g[remove_idx, :]  = g2

    return g.T

In [ ]:
num_labeled   = np.shape(tr_X)[0]
num_unlabeled = np.shape(UX)[0]
num_all       = num_labeled + num_unlabeled

x_dim        = np.shape(tr_X)[1]   
y_dim        = np.shape(tr_Y_onehot)[1]

y_type = 'categorical'

# STEP1: SELF-SUPERVISION PHASE

In [ ]:
reg_scale      = 0. #1e-3 #0. #1e-1 #0. #1e-4 #0. #1e-4#0. #1e-5 #0.#1e-8 #1e-5# 0. #1e-4 

num_layers_e   = 3

h_dim_e        = 100
z_dim          = 10

input_dims = {
    'x_dim': x_dim,
    'z_dim': z_dim
} 


network_settings = {
    'h_dim_e': h_dim_e,
    'num_layers_e': num_layers_e,
    'h_dim_d': h_dim_e,
    'num_layers_d': num_layers_e,
    
    'fc_activate_fn': tf.nn.relu, #tf.nn.tanh, #tf.nn.relu,
    'reg_scale': reg_scale
}

In [ ]:
tf.reset_default_graph()

# Turn on xla optimization
config = tf.ConfigProto()
# config = tf.ConfigProto(device_count = {'GPU': 0})
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
model = SEFS_SS_Phase(sess, "pretraining", input_dims, network_settings)

In [ ]:
step_size      = 1000
iteration      = 500000

mb_size        = 32
learning_rate  = 1e-4

keep_prob      = 1.0
alpha          = 10.0

p = 0.5


x_mean = np.mean(UX, axis=0, keepdims=True)

UX2 = UX[:1000]
UX  = UX[1000:]

In [ ]:
sess.run(tf.global_variables_initializer())
saver       = tf.train.Saver()

In [ ]:
print('=============================================')
print('Start Feature Selection .... OUT_ITR {}, ALPHA {}, #MB {}, KEEP PROB {}'.format(out_itr, alpha, mb_size, keep_prob))
print('=============================================')

avg_loss     = 0
va_avg_loss  = 0 

avg_loss_r1    = 0
va_avg_loss_r1 = 0 

avg_loss_r2    = 0
va_avg_loss_r2 = 0 

max_acc      = 0.    
min_loss     = 1e+8

max_flag     = 20
stop_flag    = 0

for itr in range(iteration):       
    x_mb, _       = f_get_minibatch(mb_size, UX, UX)
    x2_mb         = np.tile(x_mean, [np.shape(x_mb)[0], 1])

    m_mb          = mask_generation(x_mb.shape[0], p)

    _, tmp_loss, tmp_loss_r1, tmp_loss_r2  = model.train_main(
        x_=x_mb, x_bar_=x2_mb, m_=m_mb, alpha_=alpha, lr_train_=learning_rate, k_prob_=keep_prob
    )

    avg_loss      += tmp_loss/step_size
    avg_loss_r1   += tmp_loss_r1/step_size
    avg_loss_r2   += tmp_loss_r2/step_size

    
    x_mb, _       = f_get_minibatch(min(mb_size, np.shape(UX2)[0]), UX2, UX2)
    x2_mb         = np.tile(x_mean, [np.shape(x_mb)[0], 1])

    m_mb          = mask_generation(x_mb.shape[0], p)

    tmp_loss, tmp_loss_r1, tmp_loss_r2   = model.get_loss_main(x_=x_mb, x_bar_=x2_mb, m_=m_mb, alpha_=alpha)
    va_avg_loss     += tmp_loss/step_size
    va_avg_loss_r1  += tmp_loss_r1/step_size
    va_avg_loss_r2  += tmp_loss_r2/step_size
            
    if (itr+1)%step_size == 0:
        stop_flag += 1
        
        print("ITR {:05d}  | TR: loss={:.3f} loss_Rx={:.3f} loss_Rm={:.3f}  | VA: loss={:.3f} loss_Rx={:.3f} loss_Rm={:.3f}".format(
            itr+1, avg_loss, avg_loss_r1, avg_loss_r2, va_avg_loss, va_avg_loss_r1, va_avg_loss_r2
        ))
        
        
        if va_avg_loss < min_loss:
            print('saved...')
            saver.save(sess, save_path + 'model_pretrained')
            np.savez(save_path + 'model_pretrained_encoder.npz', *sess.run(model.vars_encoder))
            
            min_loss  = va_avg_loss
            
            stop_flag = 0
            
        avg_loss     = 0
        va_avg_loss  = 0 

        avg_loss_r1     = 0
        va_avg_loss_r1  = 0 
        
        avg_loss_r2     = 0
        va_avg_loss_r2  = 0 
        
        if stop_flag >= max_flag:
            break

# STEP2: SUPERVISION PHASE

In [ ]:
reg_scale      = 0. 

num_layers_p   = 1
h_dim_p        = 100


input_dims = {
    'x_dim': x_dim,
    'z_dim': z_dim,
    'y_dim': y_dim,
    'y_type': y_type
} 


network_settings = {
    'h_dim_e': h_dim_e,
    'num_layers_e': num_layers_e,
    'h_dim_p': h_dim_p,
    'num_layers_p': num_layers_p,
    
    'fc_activate_fn_e': tf.nn.relu, 
    'fc_activate_fn_p': tf.nn.relu, 
    
    'reg_scale': reg_scale
}

In [ ]:
tf.reset_default_graph()

# Turn on xla optimization
config = tf.ConfigProto()
# config = tf.ConfigProto(device_count = {'GPU': 0})
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
model = SEFS_S_Phase(sess, "feature_selection", input_dims, network_settings)

In [ ]:
step_size      = 1000
iteration      = 500000

In [ ]:
mb_size        = 32
mb_size        = min(mb_size, np.shape(tr_X)[0])
 
learning_rate  = 1e-4

keep_prob      = 1.0 
lmbda          = 1.0 

In [ ]:
sess.run(tf.global_variables_initializer())
saver       = tf.train.Saver()

In [ ]:
pretrained_encoder = np.load(save_path + 'model_pretrained_encoder.npz', allow_pickle=True)

for i in range(len(list(pretrained_encoder))):
    sess.run(tf.assign(model.vars_encoder[i], pretrained_encoder[list(pretrained_encoder)[i]]))

In [ ]:
va_X        = np.copy(tr_X)
va_Y_onehot = np.copy(tr_Y_onehot)

In [ ]:
print('=============================================')
print('Start Feature Selection .... OUT_ITR {}, LAMBDA {}, KEEP PROB {}'.format(out_itr, lmbda, keep_prob))
print('=============================================')

avg_loss      = 0.
avg_loss_m0   = 0.   

va_avg_loss      = 0.
va_avg_loss_m0   = 0.

max_auc      = 0.    
min_loss     = 1e+8

max_flag     = 20
stop_flag    = 0

num_selected_curr = 0
num_selected_prev = 0


for itr in range(iteration):
    x_mb, y_mb     = f_get_minibatch(min(mb_size, np.shape(tr_X)[0]), tr_X, tr_Y_onehot)
    x2_mb          = np.tile(x_mean, [np.shape(x_mb)[0], 1])
    q_mb           = copula_generation(mb_size)
    
    _, tmp_loss, tmp_loss_m0  = model.train_finetune(x_=x_mb, x_bar_=x2_mb, y_=y_mb, q_=q_mb, lmbda_=lmbda, lr_train_=learning_rate, k_prob_=keep_prob)
    avg_loss      += tmp_loss/step_size
    avg_loss_m0   += tmp_loss_m0/step_size
    

    tmp_loss, tmp_loss_m0     = model.get_loss(x_=x_mb, x_bar_=x2_mb, y_=y_mb, q_=q_mb, lmbda_=lmbda)
    
    va_avg_loss      += tmp_loss/step_size
    va_avg_loss_m0   += tmp_loss_m0/step_size    
                
    
    if (itr+1)%step_size == 0:
        stop_flag  += 1

        tmp_mask = (sess.run(model.pi) > 0.5).astype(float)
        q_mb     = copula_generation(np.shape(va_X)[0])
        
        tmp_y    = model.predict(x_=va_X, x_bar_=np.tile(x_mean, [np.shape(va_X)[0], 1]), q_=q_mb)
        tmp_y2   = model.predict_final(x_=va_X, x_bar_=np.tile(x_mean, [np.shape(va_X)[0], 1]), m_=tmp_mask)
        
        va_auc, va_apc   = cal_metrics(va_Y_onehot, tmp_y)
        va_auc2, va_apc2 = cal_metrics(va_Y_onehot, tmp_y2)

        print("ITR {:05d}  | TR: loss={:.3f} loss_m0={:.3f}  | VA: loss={:.3f} loss_m0={:.3f} AUC:{:.3f}, AUC_Selected:{:.3f}".format(
            itr+1, avg_loss, avg_loss_m0, va_avg_loss, va_avg_loss_m0, va_auc, va_auc2
        ))
        


        if va_avg_loss < min_loss:
            print('saved...')
            saver.save(sess, save_path + 'sefs_trained')
            
            min_loss  = va_avg_loss
            
            stop_flag = 0


        avg_loss      = 0.
        avg_loss_m0   = 0.   

        va_avg_loss      = 0.
        va_avg_loss_m0   = 0.

        if stop_flag >= max_flag:
            break

In [ ]:
feature_importance = sess.run(model.pi)

In [ ]:
plt.bar(range(x_dim), feature_importance) 
plt.bar(range(0, x_dim, blocksize), feature_importance[range(0, x_dim, blocksize)])  # x0, x10, x20, x30, ...


plt.xlabel('Feature Number', fontsize=12)
plt.ylabel('Feature Importance', fontsize=12)
plt.show()
plt.close()